# This notebook file is used aligning fitbit for the PA protocol

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, time, timedelta
from matplotlib import pyplot as plt

## The block of code below reads in the fitbit-hr and fitabase. Then it plots them
<p>Please be sure to change the first line of code in the block below. You should change the participant number to the current participant that you are trying to process. For example the end of the line may read something like '\\2989\\'. If you wanted to process participant 3000 you would change the 2989 to 3000.</p>

In [2]:
path = "V:\\ACOI\\R01 - W4K\\1_Sleep Study\\1_Participant Data\\7517032223\\"
part_id = path[-11:-1]
# Read in tracking sheet to obtain protocol date
tracking_sheet = pd.read_excel("V:\\ACOI\\R01 - W4K\\1_Sleep Study\\Sleep study tracking.xlsx").dropna(how='all')
times = tracking_sheet.loc[tracking_sheet['Child ID'] == int(part_id), ['Date of Visit', 'Devices time on', 'Devices time off']].iloc[0]
start = datetime.combine(times.iloc[0], times.iloc[1])
end = datetime.combine(times.iloc[0] + timedelta(days=1), times.iloc[2])
print(f"Protocol Start: {start} \nProtocol End: {end}")

# Read in FITBIT data
slog_hr = pd.read_csv(path + "Fitbit\\" + part_id + "_heart.csv", parse_dates=['Time'], infer_datetime_format=True)
slog_hr

# Read in FITABASE data
fitabase = pd.read_csv(path + "Fitbit\\Fitabase\\" + part_id + "_hr.csv", parse_dates=['Time'])
# Select data corresponding to protocol
#fitabase = fitabase.loc[(fitabase['Time'] >= start) & (fitabase['Time'] <= end), :].reset_index(drop=True)
fitabase = fitabase.loc[(fitabase['Time'] >= start - timedelta(minutes=40)) & (fitabase['Time'] <= end + timedelta(hours=3)), :].reset_index(drop=True)
fitabase

# Plot HR to check initial alignment
fig, ax = plt.subplots(figsize=[25,15])
ax.plot(slog_hr['Time'], slog_hr['Heart Rate'], label='Fitbit', color='red')
ax.plot(fitabase['Time'], fitabase['Value'], label='Fitabase', color='orange')
#ax.set(xlim=[start, end])
#ax.set(xlim=[start, start + timedelta(hours=2)])
fig.legend(fontsize='xx-large')
plt.show()

Protocol Start: 2023-03-22 22:40:00 
Protocol End: 2023-03-23 05:17:00


## The code block below attempts to align the fitbit and fitabase using the hr peaks.
<p>This doesn't always work. If it doesn't work you will then need to run the next code block. If it does work, skip to the very last code block</p>

In [6]:
## The code block below attempts to align the fitbit and fitabase using the hr peaks.
<p>This doesn't always work. If it doesn't work you will then need to run the next code block. If it does work, skip to the very last code block</p># OPTIONAL
# Align FITBIT with FITABASE
# First check when Max's occure
slog_max = slog_hr.iloc[slog_hr['Heart Rate'].idxmax(), 0]
fitabase_max = fitabase.iloc[fitabase['Value'].idxmax(), 0]
print(f"Fitbit Max Occured: {slog_max}\nFitabase Max Occured: {fitabase_max}")

# Shift data
if fitabase_max > slog_max :
    shift = fitabase_max - slog_max
    slog_hr['Time'] = slog_hr['Time'].apply(lambda x: x + shift)
else :
    shift = slog_max - fitabase_max
    slog_hr['Time'] = slog_hr['Time'].apply(lambda x: x - shift)

# Plot HR to check initial alignment
fig, ax = plt.subplots(figsize=[25,15])
ax.plot(slog_hr['Time'], slog_hr['Heart Rate'], label='Fitbit', color='red')
ax.plot(fitabase['Time'], fitabase['Value'], label='Fitabase', color='orange')
#ax.set(xlim=[start, end])
#ax.set(xlim=[start, start + timedelta(hours=2)])
fig.legend(fontsize='xx-large')
plt.show()

Fitbit Max Occured: 2023-04-04 01:48:03.183000
Fitabase Max Occured: 2023-04-04 02:57:49


## IF the above block did not align the fitbit and the fitabase, try the block below.
<p>You will once again need to change the first line of code. The first line of code is adding (or subtracting) some time to all the times in the fitbit time column. You can adjust whether your adding or subracting and how much time your adding/subtracting</p>

In [11]:
slog_hr['Time'] = slog_hr['Time'].apply(lambda x: x - timedelta(seconds=30))

# Plot to check alignment
fig, ax = plt.subplots(figsize=[25,15])
ax.plot(slog_hr['Time'], slog_hr['Heart Rate'], label='Fitbit', color='red')
ax.plot(fitabase['Time'], fitabase['Value'], label='Fitabase', color='orange')
ax.set(xlim=[start, end])
#ax.set(xlim=[start + timedelta(hours=2), start + timedelta(hours=4)])
fig.legend(fontsize='xx-large')
plt.show()

## Run the last line of code to save the changes you've made to the fitbit heartrate file

In [14]:
# Write FITBIT data to CSV
slog_hr.to_csv(path + "Fitbit\\" + part_id+ "_heart.csv", index=None)